In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
SCOPETYPE = 'OPENvADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'AVRCRYPTOLIB'

In [ ]:
def load_bitstream(bitstream_file):
    %run "../Setup_Scripts/Setup_Generic.ipynb"
    scope.scopetype.cwFirmwareConfig[0xACE2].loader.setFPGAMode("debug")
    scope.scopetype.cwFirmwareConfig[0xACE2].loader._bsLoc = bitstream_file
    scope.scopetype.cwFirmwareConfig[0xACE2].loader.save_bsLoc()
    print("Mode: " + str(scope.scopetype.cwFirmwareConfig[0xACE2].loader._release_mode))
    print("Mode: " + str(scope.scopetype.cwFirmwareConfig[0xACE2].loader.fpga_bitstream()))
    input("powercycle done?")
    %run "../Setup_Scripts/Setup_Generic.ipynb"
    
load_bitstream("../../hardware/capture/chipwhisperer-lite/cwlite_interface_ec_256.bit")

In [ ]:
import holoviews as hv
import pandas as pd
import numpy as np

from holoviews.streams import Pipe, Buffer
from scipy.ndimage.filters import uniform_filter1d
from tqdm.notebook import trange

import chipwhisperer.analyzer as cwa
import chipwhisperer as cw
from chipwhisperer.common.traces import Trace
import edge_counter

class real_time_plot:
    def __init__(self, plot_len, title="Plot"):
        hv.extension('bokeh')
        st = pd.DataFrame({'y':[]}, columns=['y'])
        self.dfstream = Buffer(st, length=plot_len, index=False)
        curve_dmap = hv.DynamicMap(hv.Curve, streams=[self.dfstream])
        curve_dmap.opts(tools=["hover"], width=800, height=600, title=title)
        display(curve_dmap)
        self.curve = curve_dmap
    def send(self, data):
        if hasattr(data, 'wave'):
            d = data.wave
        else:
            d = data
        self.dfstream.send(pd.DataFrame({'y':d}))

def powercycle_target(scope):
    scope.io.tio1 = "gpio_low"
    time.sleep(0.0001)
    scope.arm()
    scope.io.tio1 = "gpio_high"
    
def print_cw_conf():
    print("==== EC ====")
    print(scope.EC)
    print("==== trigger ====")
    print(scope.trigger)
    print("==== adc ====")
    print(scope.adc)
    print("============")
    
def setup_cw():
    scope.default_setup()
    scope.adc.offset = offset
    scope.gain.gain = 50
    scope.adc.samples = nr_samples
    #scope.adc.presamples = 400
    
    # per docs has to be like that for DecodeIO and SAD
    scope.adc.basic_mode = "rising_edge"
    
def setup_glitcher():
    scope.glitch.width = 0.2
    scope.glitch.width_fine = 200
    scope.glitch.trigger_src = "ext_single"
    scope.glitch.repeat = 1
    scope.glitch.output = "glitch_only"

def setup_ec(nb_win_size, nb_threshold, nb_edge_type, nb_pretrigger_ctr, nb_edge_num):
    setup_cw()
    scope.EC.reset()
    scope.EC.window_size = nb_win_size
    scope.EC.threshold = nb_threshold
    scope.EC.hold_cycles = nb_pretrigger_ctr
    scope.EC.absolute_values = True
    scope.EC.edge_num = nb_edge_num
    scope.EC.edge_type = nb_edge_type
    scope.EC.start()
    scope.trigger.module = "EC"

def collect_trace():
    
    powercycle_target(scope)
    scope.capture()
    return scope.get_last_trace()
    
# init plotting        
hv.extension('bokeh')

nr_samples = 24400  
offset = 0
traces = 5

nb_win_size = 200
nb_threshold = 0.014
nb_edge_type = "rising_edge"
nb_pretrigger_ctr = 20
nb_edge_num = 1
scope.adc.decimate = 80
scope.adc.presamples = 0

setup_ec(nb_win_size, nb_threshold, nb_edge_type, nb_pretrigger_ctr, nb_edge_num)

ec_offline = edge_counter.edge_count(nb_win_size, nb_threshold, nb_edge_num, nb_edge_type, nb_pretrigger_ctr)

print_cw_conf()

In [ ]:
plot = real_time_plot(nr_samples)
traces = []
for i in range(0, 20):
    trace = collect_trace()
    traces.append(trace)
    plot.send(trace)

Offline calculation for above traces

In [ ]:
# trace = collect_trace()
i = 0
orig_curve = hv.Curve(traces[i]).opts(tools=["hover"], width=500, height=600, title="orig", axiswise=True)
trace_sum = ec_offline._movavg(traces[i])
movsum_curve = hv.Curve(trace_sum).opts(tools=["hover"], width=500, height=600, title="avg", axiswise=True)
(orig_curve)+(movsum_curve)

# Collect and save traces

In [ ]:
import chipfail_lib
import serial

def setup_basic():
    setup_cw()

    scope.adc.basic_mode = "rising_edge"
    scope.trigger.module = "basic"
    scope.trigger.triggers = "tio4"

def collect_trace_basic():
    scope.arm()
    
    # As long as only the A7's power port is connected to reset this will issue a 1us reset
    # The CW can trigger on the reset release
    chipfail_lib.manual_glitch(fpga)
    
    scope.capture()
    return scope.get_last_trace()

# Show a longer trace to get a sense of what's going on in the target
def collect_long_trace(traces, nr_samples, offset):
    total_len = traces*nr_samples

    new_trace = np.zeros(0)
    for i in trange(traces, desc='Capturing traces'):
        cur_offset = int(offset + nr_samples*i)
        scope.adc.offset = cur_offset

        trace = collect_trace_basic()
        
        if trace is None:
            continue
        new_trace = np.append(new_trace, trace)
    
    return new_trace

def collect_interpolate_traces(traces, nr_samples, offset, interpolate=True):
    long_trace = collect_long_trace(traces, nr_samples, offset)

    orig_curve = hv.Curve(long_trace).opts(tools=["hover"], width=800, height=600, title="orig", axiswise=True)
    if interpolate:
        trace_avg, trig_avg = ec_offline.run(long_trace, "avg")
        trace_sum, trig_sum = ec_offline.run(long_trace, "sum")

        print(f"trig_avg: {trig_avg}")

        movsum_curve = hv.Curve(trace_sum).opts(tools=["hover"], width=800, height=600, title="sum", axiswise=True)
        movavg_curve = hv.Curve(trace_avg).opts(tools=["hover"], width=800, height=600, title="avg", axiswise=True)

        return (orig_curve, movsum_curve, movavg_curve)
    else:
        return (orig_curve, None, None)

def build_project(name):
    project = cw.create_project(name)

    for i in range(0, 9):
        long_trace = collect_reference_trace(traces, nr_samples, offset)
        project.traces.append(Trace(long_trace, None, None, None))

    project.save()

def display_project(name):
    project = cw.open_project(name)
    
    for trace in project.traces:
        pass

nr_samples = 24400  
offset = 0
traces = 70

nb_win_size = 200
nb_threshold = 0.008
nb_edge_type = "rising_edge"
nb_pretrigger_ctr = 10
nb_edge_num = 1
scope.adc.decimate = 80
scope.adc.presamples = 0

setup_basic()
# print_cw_conf()
ec_offline = edge_counter.edge_count(nb_win_size, nb_threshold, nb_edge_num, nb_edge_type, nb_pretrigger_ctr)
# setup_ec(nb_win_size, nb_threshold, nb_edge_type, nb_pretrigger_ctr, nb_edge_num)

# Initialize connection to ARTY A7 FPGA
fpga = serial.Serial("/dev/ttyUSB2", baudrate=115200)
chipfail_lib.setup(fpga)

proj = cw.open_project("projects/recovery")
all_traces = {}
for i in trange(250, 260):
    trace = collect_trace_basic()
    if i > 250 and i < 256:
        orig_curve = hv.Curve(trace).opts(tools=["hover"], width=800, height=600, title="orig", axiswise=True)
#     orig_curve, _, _ = collect_interpolate_traces(traces, nr_samples, offset, False)
        all_traces[i] = orig_curve
    proj.traces.append(Trace(trace, None, None, None))
    
proj.save()
    
ndoverlay = hv.NdOverlay(all_traces).opts(legend_limit=1)
ndoverlay

In [ ]:
mb2_proj = cw.open_project("projects/working_mb2")
mb1_proj = cw.open_project("projects/invalid_mb1")

mb2_traces = {}
mb1_traces = {}
for i in range(350, 360):
    mb2_traces[i] = hv.Curve(mb2_proj.traces[i].wave).opts(tools=["hover"], width=800, height=600, title="orig", axiswise=False)
    mb1_traces[i] = hv.Curve(mb1_proj.traces[i].wave).opts(tools=["hover"], width=800, height=600, title="orig", axiswise=False)

mb2_overlay = hv.NdOverlay(mb2_traces).opts(legend_limit=1)
mb1_overlay = hv.NdOverlay(mb1_traces).opts(legend_limit=1)

mb2_overlay+mb1_overlay

In [ ]:
len(proj.traces)

In [ ]:
hv.help(hv.NdOverlay)